In [1]:
import pandas as pd
import string

In [16]:
# load the data for comparision

df1 = pd.read_csv('Menu.csv')
df2 = pd.read_csv('cleanMenu.csv')

In [17]:
df2['date'] = pd.to_datetime(df2['date'], errors='coerce')

df2 = df2[
    (df2['date'].notna()) & (df2['date'].dt.year <= 2024) &
    (df2['status'].isin(['complete', 'under review'])) &
    (df2['page_count'] <= 100) &
    (df2['dish_count'] <= 1000)
]

In [18]:
df1i = df1
df2i = df2

df1i.set_index('id', inplace=True)
df2i.set_index('id', inplace=True)

sponsor_before = df1i[['sponsor']]
sponsor_after = df2i[['sponsor']]

all_indices = sponsor_before.index.union(sponsor_after.index)
sponsor_before_aligned = sponsor_before.reindex(index=all_indices).fillna('MISSING')
sponsor_after_aligned = sponsor_after.reindex(index=all_indices).fillna('MISSING')

comparison = sponsor_before_aligned != sponsor_after_aligned
num_changed_cells = comparison.sum().sum()

removed_indices = sponsor_before.index.difference(sponsor_after.index)
num_removed_cells = len(removed_indices)

print(f"Number of changed cells in 'sponsor' column: {num_changed_cells}")
print(f"Number of removed cells in 'sponsor' column: {num_removed_cells}")

changed_indices = comparison[comparison['sponsor']].index
for idx in changed_indices[:20]:
    before_value = sponsor_before_aligned.at[idx, 'sponsor']
    after_value = sponsor_after_aligned.at[idx, 'sponsor']
    print(f"ID: {idx}, Before: {before_value}, After: {after_value}")

Number of changed cells in 'sponsor' column: 10431
Number of removed cells in 'sponsor' column: 2210
ID: 12464, Before: REPUBLICAN HOUSE, After: REPUBLIC HOUSE
ID: 12469, Before: HOTEL NETHERLAND, After: NETHERLAND HOTEL
ID: 12484, Before: BOSTON BOOT & SHOE CLUB, After: BOSTON BOOT AND SHOE CLUB
ID: 12486, Before: HOTEL SAVOY, After: SAVOY HOTEL
ID: 12487, Before: PACIFIC MAIL STEAMSHIP CO., After: PACIFIC MAIL STEAMSHIP COMPANY
ID: 12490, Before: HOTEL IMPERIAL, After: IMPERIAL HOTEL
ID: 12499, Before: PACIFIC MAIL STEAMSHIP CO., After: PACIFIC MAIL STEAMSHIP COMPANY
ID: 12507, Before: HAAN'S, After: HAANS
ID: 12509, Before: RED STAR LINE - S.S.SOUTHWARK, After: RED STAR LINE - S.S. SOUTHWARK
ID: 12511, Before: THIRD PANEL SHERIFF'S JURY NEW YORK COUNTY, After: THIRD PANEL SHERIFFS JURY NEW YORK COUNTY
ID: 12520, Before: BAILY CATERING CO.THE, After: BAILEY CATERING COMPANY
ID: 12523, Before: CITIZENS'STEAMBOAT COMPANY, After: CITIZENSSTEAMBOAT COMPANY
ID: 12539, Before: GOULD'S HOTE

In [19]:
# check how the number nan and empty 

num_nan1 = df1['sponsor'].isna().sum()
num_empty1 = (df1['sponsor'] == '').sum()
num_nan2 = df2['sponsor'].isna().sum()
num_empty2 = (df2['sponsor'] == '').sum()

print(f'Original: Number of NaN cells: {num_nan1}')
print(f'Original: Number of empty cells: {num_empty1}')
print(f'Modified: Number of NaN cells: {num_nan2}')
print(f'Modified: Number of empty cells: {num_empty2}')

Original: Number of NaN cells: 1561
Original: Number of empty cells: 0
Modified: Number of NaN cells: 0
Modified: Number of empty cells: 0


In [20]:
# check if any cells are not either lower case, upper case or title case

def is_not_case_compliant(text):
    if pd.isna(text):  # Treat NaN values as compliant
        return False
    text = str(text)
    return not (text.isupper() or text.islower() or text.istitle())

num_non_compliant1 = df1['sponsor'].apply(is_not_case_compliant).sum()
num_non_compliant2 = df2['sponsor'].apply(is_not_case_compliant).sum()

print(f'Original: Number of cells not in upper case, lower case, or title case: {num_non_compliant1}')
print(f'Modified: Number of cells not in upper case, lower case, or title case: {num_non_compliant2}')

Original: Number of cells not in upper case, lower case, or title case: 1404
Modified: Number of cells not in upper case, lower case, or title case: 0


In [21]:
# check how many unique cells

num_unique_values1 = df1['sponsor'].nunique()
num_unique_values2 = df2['sponsor'].nunique()

print(f'Original: Number of unique values in the sponsor column: {num_unique_values1}')
print(f'Modified: Number of unique values in the sponsor column: {num_unique_values2}')

Original: Number of unique values in the sponsor column: 6370
Modified: Number of unique values in the sponsor column: 5111


In [22]:
# check the sponsors that have multiple location

location_counts1 = df1.groupby('sponsor')['location'].nunique()
multiple_locations1 = location_counts1[location_counts1 > 1]
num_sponsors_multiple_locations1 = multiple_locations1.count()

location_counts2 = df2.groupby('sponsor')['location'].nunique()
multiple_locations2 = location_counts2[location_counts2 > 1]
num_sponsors_multiple_locations2 = multiple_locations2.count()

print(f'Original: Number of sponsors with multiple locations: {num_sponsors_multiple_locations1}')
print(f'Modified: Number of sponsors with multiple locations: {num_sponsors_multiple_locations2}')

print("\nOriginal: Sponsors with multiple locations and their count of unique locations:")
print(multiple_locations1)

print("\nModified: Sponsors with multiple locations and their count of unique locations:")
print(multiple_locations2)

Original: Number of sponsors with multiple locations: 93
Modified: Number of sponsors with multiple locations: 508

Original: Sponsors with multiple locations and their count of unique locations:
sponsor
(AMERICAN LINE)                                 2
?                                              15
ALCAZAR HOTEL                                   2
BALTIMORE AND OHIO RR ROYAL BLUE LINE           2
BELVEDERE HOUSE;                                2
                                               ..
Unknown                                         3
VENDOME HOTEL                                   2
Waldorf-Astoria                                 2
[Restaurant And/Or Location Unknown]            2
[Restaurant Name And/Or Location Not Given]     9
Name: location, Length: 93, dtype: int64

Modified: Sponsors with multiple locations and their count of unique locations:
sponsor
A.H. MEYER RATHSKELLER                               2
A.W. DENNETT                                         2
ADAMS 

In [23]:
sort_locations1 = location_counts1.sort_values(ascending=False)
print(f'Original: The most popular sponsor is:\n\n {sort_locations1}') # .to_string(header=False)
sort_locations2 = location_counts2.sort_values(ascending=False)
print(f'\n\nModified The most popular sponsor is:\n\n {sort_locations2}')

Original: The most popular sponsor is:

 sponsor
?                                              15
[Restaurant Name And/Or Location Not Given]     9
CONFRERIE DE LA CHAINE DES ROTISSEURS           5
USMS                                            4
U.S.M.S.                                        4
                                               ..
HOTEL DE SUN                                    1
HOTEL DE L'ANGE                                 1
HOTEL DE DIJON                                  1
HOTEL DE BELLEVUE                               1
xtYE TWILIGHT CLUB                              1
Name: location, Length: 6370, dtype: int64


Modified The most popular sponsor is:

 sponsor
NORDDEUTSCHER LLOYD BREMEN                   19
RESTAURANT NAME AND/OR LOCATION NOT GIVEN    15
RED STAR LINE - ANTWERPEN - NY               10
U.S.M.S.                                      9
HAMBURG-AMERIKA LINIE                         7
                                             ..
HOTEL CADILLAC      

In [24]:
from datetime import datetime
current_date = datetime.now()

df1['date'] = pd.to_datetime(df1['date'], errors='coerce')

date_violations = df1[df1['date'] > current_date]
status_violations = df1[~df1['status'].isin(['complete', 'under review'])]
page_count_violations = df1[df1['page_count'] > 100]
dish_count_violations = df1[df1['dish_count'] > 4000]
# a = df1[df1['id'] == 26825]
# print(a)
print("Before cleaning:")
print(f"Number of rows with date greater than current date: {len(date_violations)}")
print(f"Number of rows with invalid status: {len(status_violations)}")
print(f"Number of rows with page count greater than 100: {len(page_count_violations)}")
print(f"Number of rows with dish count greater than 4000: {len(dish_count_violations)}")

violations = {
    'date_greater_than_current': df2[df2['date'] > current_date],
    'invalid_status': df2[~df2['status'].isin(['complete', 'under review'])],
    'page_count_greater_than_100': df2[df2['page_count'] > 100],
    'dish_count_greater_than_1000': df2[df2['dish_count'] > 4000]
}

print("After cleaning:")
print(f"Number of rows with date greater than current date: {len(violations['date_greater_than_current'])}")
print(f"Number of rows with invalid status: {len(violations['invalid_status'])}")
print(f"Number of rows with page count greater than 100: {len(violations['page_count_greater_than_100'])}")
print(f"Number of rows with dish count greater than 4000: {len(violations['dish_count_greater_than_1000'])}")

Before cleaning:
Number of rows with date greater than current date: 0
Number of rows with invalid status: 0
Number of rows with page count greater than 100: 0
Number of rows with dish count greater than 4000: 1
After cleaning:
Number of rows with date greater than current date: 0
Number of rows with invalid status: 0
Number of rows with page count greater than 100: 0
Number of rows with dish count greater than 4000: 0


In [25]:
df2.to_csv('cleanMenu.csv')